In [1]:
import tqdm
import numpy as np
import torch
import pickle
import os

from libero.libero import benchmark
from libero.libero.envs import OffScreenRenderEnv

from IPython.display import display
from PIL import Image

from libero.libero.benchmark import (
    register_benchmark,
    grab_language_from_filename,
    Benchmark,
    Task,

)


In [2]:
generate_pddl = False
generate_init_states = True
suite_name = "libero_10_diff_obj"


num_init_states = 5
num_pruned_states = 2

In [3]:
######## Generate PDDL file ###########

# Currenty just changing objects that are manipulated, we can just copy the previous pddl files

In [4]:
######## Generate Init States #########

# Currenty just changing objects that are manipulated, we can just copy the previous init files

if generate_init_states:
    # TODO: allow multiiple tasks to be generated
    task = "LIVING_ROOM_SCENE2_put_orange_juice_in_the_basket"
    env_args = {
        "bddl_file_name": f"/home/leisongao/LIBERO/notebooks/custom_pddl/{task}.bddl",
        "camera_heights": 256,
        "camera_widths": 256
    }

    env = OffScreenRenderEnv(**env_args)

    init_states = []
    for i in tqdm.tqdm(range(num_init_states), desc=f"Sampling init states for task: {task}"):
        env.seed(i)
        obs = env.reset()
        init_state = env.sim.get_state().flatten()
        init_states.append(init_state)
    
    init_states = np.array(init_states)
    pruned_states = init_states[:num_pruned_states]

    data = pickle.dumps(init_states)
    pruned_data = pickle.dumps(pruned_states)

    # Save in LIBERO format (npz container with 'archive/data.pkl')
    archive = {
        "archive/data.pkl": np.array(data),
        "archive/version": b"1\n", # this is not exact format because of np.savez wraps in np.array
    }

    pruned_archive = {
        "archive/data.pkl": np.array(pruned_data),
        "archive/version": b"1\n",
    }

    # TODO: change to be for save directory and automatically
    # save_dir = f"/home/leisongao/LIBERO/libero/libero/init_files/{suite_name}"
    save_dir = f"/home/leisongao/vlarl/LIBERO/libero/libero/./init_files/{suite_name}"
    
    os.makedirs(save_dir, exist_ok=True)

    save_path = os.path.join(save_dir, task)

    torch.save(np.array(data), save_path + ".init")
    # np.savez(save_path, **archive)
    # os.rename(save_path + ".npz", save_path + ".init")

    torch.save(np.array(pruned_data), save_path + ".pruned_init")

    # # TODO: possible aliasing issue here? don't care enough atm
    # np.savez(save_path, **pruned_archive)
    # os.rename(save_path + ".npz", save_path + ".pruned_init")

    print(f"Saved init files in {save_path}")

Sampling init states for task: LIVING_ROOM_SCENE2_put_orange_juice_in_the_basket: 100%|██████████| 5/5 [00:04<00:00,  1.12it/s]

Saved init files in /home/leisongao/vlarl/LIBERO/libero/libero/./init_files/libero_10_diff_obj/LIVING_ROOM_SCENE2_put_orange_juice_in_the_basket


In [5]:
######## Create Task Suite ##########
task_map = {}
libero_suite = "libero_10_diff_obj"
tasks = [
    "LIVING_ROOM_SCENE2_put_orange_juice_in_the_basket",
]

for task in tasks:
    language = grab_language_from_filename(task + ".bddl")
    task_map[task] = Task(
        name=task,
        language=language,
        problem="Libero",
        problem_folder=libero_suite,
        bddl_file=f"{task}.bddl",
        init_states_file=f"{task}.pruned_init",
    )


In [6]:

class LIBERO_10_diff_obj(Benchmark):
    def __init__(self, task_order_index=0):
        super().__init__(task_order_index=task_order_index)
        self.name = "libero_10_diff_obj"
        self._make_benchmark() 
    
    def _make_benchmark(self):
        tasks = list(task_map.values())
        self.tasks = tasks
        # not using task orders as of now (we want to be able to control the curriculum, but do this later )
        # print(f"[info] using task orders {task_orders[self.task_order_index]}")
        # self.tasks = [tasks[i] for i in task_orders[self.task_order_index]]
        self.n_tasks = len(self.tasks)

# Register it explicitly
register_benchmark(LIBERO_10_diff_obj)


In [7]:
benchmark_dict = benchmark.get_benchmark_dict(help=True)

Available benchmarks:
	libero_spatial
	libero_object
	libero_goal
	libero_90
	libero_10
	libero_100
	libero_10_diff_obj


In [8]:
task_suite = benchmark_dict["libero_10_diff_obj"]()

In [9]:
task_id = 0

task = task_suite.get_task(task_id)

# Get initial states
initial_states = task_suite.get_task_init_states(task_id)

# # Initialize environment and get task description
# env, task_description = get_libero_env(task, cfg.model_family, resolution=cfg.env_img_res)


[Warning]: datasets path /home/leisongao/vlarl/LIBERO/libero/libero/../datasets does not exist!


/home/leisongao/vlarl/LIBERO/libero/libero/benchmark/__init__.py:164: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  init_states = torch.load(init_states_path)


In [ ]:

task_description = task.language
task_bddl_file = os.path.join(get_libero_path("bddl_files"), task.problem_folder, task.bddl_file)
env_args = {"bddl_file_name": task_bddl_file, "camera_heights": resolution, "camera_widths": resolution}
env = OffScreenRenderEnv(**env_args)
  # IMPORTANT: seed seems to affect object positions even when using fixed initial state


In [12]:
############ Generate Init States ##########
env_args = {
    "bddl_file_name": "/home/leisongao/LIBERO/notebooks/custom_pddl/LIVING_ROOM_SCENE2_put_orange_juice_in_the_basket.bddl",
    "camera_heights": 256,
    "camera_widths": 256
}

env = OffScreenRenderEnv(**env_args)

num_init_states = 1
init_states = []
for i in tqdm.tqdm(range(num_init_states)):
    env.seed(i)
    obs = env.reset()
    init_state = env.sim.get_state().flatten()
    init_states.append(init_state)

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


In [ ]:
init_states = np.array(init_states)

# Step 3. Serialize to pickle
data = pickle.dumps(init_states)

# Step 4. Save in LIBERO format (npz container with 'archive/data.pkl')
archive = {
    "archive/data.pkl": np.array(data),
    "archive/version": np.array(1),
}

save_path = "my_custom.init"
np.savez(save_path, **archive)

print(f"Saved init file at {os.path.abspath(save_path)}")

In [2]:
init_file = "/home/leisongao/LIBERO/libero/libero/init_files/libero_10/LIVING_ROOM_SCENE2_put_both_the_alphabet_soup_and_the_tomato_sauce_in_the_basket.pruned_init"
# init_file = "/home/leisongao/LIBERO/libero/libero/init_files/libero_10_diff_obj/LIVING_ROOM_SCENE2_put_orange_juice_in_the_basket.init"
init_file = "test.init"

In [3]:
import numpy as np
import pickle
import torch

In [4]:
f = torch.load(init_file, weights_only=False)

In [5]:
f.shape

(50, 123)

In [40]:
a = np.random.rand(50, 123)

In [41]:
torch.save(a, "test.init")

In [21]:
f['archive/version']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [23]:
pickle.loads(f['archive/version'])

UnpicklingError: invalid load key, '3'.

In [25]:
data = pickle.loads(f['archive/data.pkl'])

In [26]:
data.shape

(50, 123)

In [8]:
data.dtype

dtype('float64')

In [7]:
import psutil, os
process = psutil.Process(os.getpid())
print(process.memory_info().rss / 1024**2, "MB")


83.4375 MB


In [1]:
from libero.libero import get_libero_path


In [2]:
get_libero_path()

TypeError: get_libero_path() missing 1 required positional argument: 'query_key'